In [1]:
from cmj.loa import models
from cmj.utils import Manutencao, run_sql
from django.conf import settings
from django.db.models import Q, F
from django.db.models.aggregates import Sum
from decimal import Decimal
import fitz
import re
m = Manutencao()
m.desativa_auto_now()
m.desativa_signals()

In [2]:
loa = models.Loa.objects.get(pk=2024)

In [3]:
"""(models.Despesa.objects.filter(loa=loa).delete(), 
models.Orgao.objects.filter(loa=loa).delete(),
models.UnidadeOrcamentaria.objects.filter(loa=loa).delete(),
models.Funcao.objects.filter(loa=loa).delete(),
models.SubFuncao.objects.filter(loa=loa).delete(),
models.Programa.objects.filter(loa=loa).delete(),
models.Acao.objects.filter(loa=loa).delete(),
models.Natureza.objects.filter(loa=loa).delete(),
models.EmendaLoaRegistroContabil.objects.filter(emendaloa__loa=loa).delete())"""

((0, {}),
 (0, {}),
 (0, {}),
 (0, {}),
 (0, {}),
 (0, {}),
 (0, {}),
 (0, {}),
 (0, {'loa.EmendaLoaRegistroContabil': 0}))

In [4]:
d = DocumentoAcessorio.objects.get(pk=3031)
settings.SITE_URL+d.arquivo.url

'https://www.jatai.go.leg.br/api/materia/documentoacessorio/3031/arquivo/'

In [5]:
dfile = fitz.open(d.arquivo.path)
#dfile = fitz.open('/home/leandro/Downloads/anexo_materia.pdf')

In [6]:
def load_pages_with(page_mask=''):
    rtext = ''
    for np in range(len(dfile)):
        page = dfile.load_page(np)
        text = page.get_text("text")

        if page_mask not in text:
            continue

        rtext += text
    while '  ' in rtext:
        rtext = rtext.replace('  ', ' ')
    while ' \n' in rtext:
        rtext = rtext.replace(' \n', '\n')
    while '\n\n' in rtext:
        rtext = rtext.replace('\n\n', '\n')
    return rtext

In [7]:
text_global = load_pages_with(page_mask='Quadro de Detalhamento da Despesa\nEXERCÍCIO DE 2024\n')

In [8]:
len(text_global), text_global[-1500:]

(68945,
 'o da Despesa\nEXERCÍCIO DE 2024\nRUA ITARUMA, 355 - S. SANTA MARIA\n60\nPÁGINA\nUNIDADE ORÇAMENTÁRIA\nÓRGÃO\nCÓDIGO\nCÓDIGO\nAPLICAÇÃO PROGRAMADA\nCÓDIGOS\nDETALHADA\nAPLICADO\nNATUREZA DA\nCLASSIFICAÇÃO ORÇAMENTÁRIA\nVALORES\nDESPESA\nTOTAL\nFEPJPEM - JATAÍ\nFUNDO ESP DOS PROCURADORES JURIDICOS PODER EXECUTIVO\n17\n30\nEQUIPAMENTOS PARA FDO PROCURADORES\n04.122.0439.1.043\n4.4.90.52.00\n3.000,00\n3.000,00\nMATENÇÃO DAS ATIV. DO FDO. PROCURADORES\n04.122.0439.2.087\n3.3.90.14.00\n1.300,00\n3.3.90.30.00\n4.000,00\n3.3.90.36.00\n888.664,89\n3.3.90.39.00\n3.335,11\n3.3.90.40.00\n800,00\n3.3.90.47.00\n1.200,00\n3.3.90.92.00\n800,00\n3.3.90.93.00\n900,00\n901.000,00\nTOTAL DA UNIDADE 30 - FUNDO ESP DOS PROCURADORES JURIDICOS PODER EXECUTIVO\n904.000,00\nTOTAL DO ÓRGÃO 17 - FEPJPEM - JATAÍ\n904.000,00\nESTADO DE GOIÁS\nMUNICÍPIO DE JATAI\nQuadro de Detalhamento da Despesa\nEXERCÍCIO DE 2024\nRUA ITARUMA, 355 - S. SANTA MARIA\n61\nPÁGINA\nUNIDADE ORÇAMENTÁRIA\nÓRGÃO\nCÓDIGO\nCÓDIGO\

In [9]:
aplicacoes = []
for m in re.finditer(r'(.+)\n(\d{2}\.\d{3}\.\d{4}\.\d\.\d{3})', text_global):
    aplicacoes.append([m.start(), m.end(), list(m.groups())])
aplicacoes.append([len(text_global),len(text_global),None])
for i in range(len(aplicacoes)-1):
    aplic = aplicacoes[i]
    x1 = aplicacoes[i][1]
    x2 = aplicacoes[i+1][0]

    natdespesas = []

    for n in re.finditer(r'(\d{1,2}\.\d{1,2}\.\d{2}\.\d{2}\.\d{2})\n(\d{0,3}(?:\.\d{3})*,\d{2})', text_global[x1:x2]):
        natdespesas.append(list(n.groups()))

    aplic.append(natdespesas)
    
aplicacoes.pop()
aplicacoes = list(map(lambda x: (x[2][1], x[2][0], x[0], x[1], tuple(x[3])), aplicacoes))

In [10]:
aplicacoes

[('01.031.0139.1.001',
  'OBRAS,INSTAL.EQUIP.CAMARA MUNICIPAL',
  329,
  382,
  (['4.4.90.51.00', '1.487.025,43'], ['4.4.90.52.00', '1.070.000,00'])),
 ('01.031.0139.2.001',
  'MANUT.ATIV.DA CAMARA MUNICIPAL',
  448,
  496,
  (['3.1.90.11.00', '17.500.000,00'],
   ['3.1.90.16.00', '4.100,00'],
   ['3.1.90.96.00', '70.000,00'],
   ['3.3.90.14.00', '75.000,00'],
   ['3.3.90.30.00', '660.000,00'],
   ['3.3.90.33.00', '1.000,00'],
   ['3.3.90.34.00', '2.500,00'],
   ['3.3.90.35.00', '5.000,00'],
   ['3.3.90.36.00', '90.000,00'],
   ['3.3.90.37.00', '10.500,00'],
   ['3.3.90.39.00', '770.000,00'],
   ['3.3.90.40.00', '160.000,00'],
   ['3.3.90.92.00', '200,00'])),
 ('01.271.2839.9.061',
  'ENC.PREV.C/O REGIME GERAL DE PREVIDÊNCIA',
  810,
  868,
  (['3.1.90.13.00', '2.100.000,00'], ['3.1.90.92.00', '200,00'])),
 ('01.272.2839.9.001',
  'ENC.PREV. C/O REGIME PRÓPRIO DE PREVIDÊNCIA',
  928,
  989,
  (['3.1.91.13.00', '1.580.000,00'],
   ['3.1.91.92.00', '200,00'],
   ['3.3.90.08.00', '200,00'

In [11]:
unidades_temp = []
for n in re.finditer(r'(?:CLASSIFICAÇÃO ORÇAMENTÁRIA\nVALORES\nDESPESA\nTOTAL\n)(.+)\n(.+)\n(.+)\n(.+)\n', text_global):
    unidades_temp.append([n.start(), n.end(), list(n.groups())])
    
unidades_temp.append([len(text_global),len(text_global),None])

In [12]:
unidades_temp[0]

[231, 329, ['CAMARA MUNICIPAL DE JATAI', 'CAMARA MUNICIPAL', '01', '01']]

In [13]:
unidades = []
for i in range(len(unidades_temp)-1):
    unidade = unidades_temp[i]
    x1 = unidades_temp[i][1]
    x2 = unidades_temp[i+1][0]
    
    unidade = [(unidade[2][2], unidade[2][0]), (unidade[2][3], unidade[2][1]), []]
    for aplic in aplicacoes:
        if aplic[2] >= x1 and aplic[3] <= x2:
            unidade[-1].append(aplic)    
    unidades.append(unidade)    

In [14]:
unidades_temp = unidades
unidades = []

orgao = ''
unidade_orc = ''
for u in unidades_temp:
    if u[0] != orgao or u[1] != unidade_orc:
        orgao = u[0]
        unidade_orc = u[1]
        unidades.append([orgao, unidade_orc, []])
    unidades[-1][-1].extend(u[2])

In [15]:
unidades

[[('01', 'CAMARA MUNICIPAL DE JATAI'),
  ('01', 'CAMARA MUNICIPAL'),
  [('01.031.0139.1.001',
    'OBRAS,INSTAL.EQUIP.CAMARA MUNICIPAL',
    329,
    382,
    (['4.4.90.51.00', '1.487.025,43'], ['4.4.90.52.00', '1.070.000,00'])),
   ('01.031.0139.2.001',
    'MANUT.ATIV.DA CAMARA MUNICIPAL',
    448,
    496,
    (['3.1.90.11.00', '17.500.000,00'],
     ['3.1.90.16.00', '4.100,00'],
     ['3.1.90.96.00', '70.000,00'],
     ['3.3.90.14.00', '75.000,00'],
     ['3.3.90.30.00', '660.000,00'],
     ['3.3.90.33.00', '1.000,00'],
     ['3.3.90.34.00', '2.500,00'],
     ['3.3.90.35.00', '5.000,00'],
     ['3.3.90.36.00', '90.000,00'],
     ['3.3.90.37.00', '10.500,00'],
     ['3.3.90.39.00', '770.000,00'],
     ['3.3.90.40.00', '160.000,00'],
     ['3.3.90.92.00', '200,00'])),
   ('01.271.2839.9.061',
    'ENC.PREV.C/O REGIME GERAL DE PREVIDÊNCIA',
    810,
    868,
    (['3.1.90.13.00', '2.100.000,00'], ['3.1.90.92.00', '200,00'])),
   ('01.272.2839.9.001',
    'ENC.PREV. C/O REGIME PRÓPRIO 

In [16]:
with open(f'funcao.txt', 'r') as fnd:
    funcoes = fnd.read()
    funcoes = re.split(r'\||\n', funcoes)
    funcoes = [(funcoes[i], funcoes[i+1]) for i in range(0, len(funcoes)-1, 2)]
    for c, e in funcoes:
        created, f = models.Funcao.objects.get_or_create(loa=loa, codigo=c, especificacao=e)
funcoes = sorted(funcoes, key=lambda x: (x[0], x[1]))
models.Funcao.objects.filter(loa=loa).count()

29

In [17]:
with open(f'subfuncao.txt', 'r') as fnd:
    subfs = fnd.read()
    subfs = re.split(r'\t|\n', subfs)
    subfs = [(subfs[i], subfs[i+1], subfs[i+2]) for i in range(0, len(subfs)-2, 3)]
    for fc, c, e in subfs:
        f = models.Funcao.objects.filter(loa=loa, codigo=fc).first()
        created, sf = models.SubFuncao.objects.get_or_create(loa=loa, funcao=f, codigo=c, especificacao=e)
subfs = sorted(subfs, key=lambda x: (x[0], x[1]))
models.SubFuncao.objects.filter(loa=loa).count()

114

In [18]:
count = [0,0,0]
for orgao, unidade, despesas in unidades:
    count[0] += 1
    for desp, espec, x1, x2, naturezas in despesas:
        count[1] += 1
        for nat in naturezas:
            count[2] += 1
print(count)

[32, 281, 1093]


In [19]:
unidades[0][2]

[('01.031.0139.1.001',
  'OBRAS,INSTAL.EQUIP.CAMARA MUNICIPAL',
  329,
  382,
  (['4.4.90.51.00', '1.487.025,43'], ['4.4.90.52.00', '1.070.000,00'])),
 ('01.031.0139.2.001',
  'MANUT.ATIV.DA CAMARA MUNICIPAL',
  448,
  496,
  (['3.1.90.11.00', '17.500.000,00'],
   ['3.1.90.16.00', '4.100,00'],
   ['3.1.90.96.00', '70.000,00'],
   ['3.3.90.14.00', '75.000,00'],
   ['3.3.90.30.00', '660.000,00'],
   ['3.3.90.33.00', '1.000,00'],
   ['3.3.90.34.00', '2.500,00'],
   ['3.3.90.35.00', '5.000,00'],
   ['3.3.90.36.00', '90.000,00'],
   ['3.3.90.37.00', '10.500,00'],
   ['3.3.90.39.00', '770.000,00'],
   ['3.3.90.40.00', '160.000,00'],
   ['3.3.90.92.00', '200,00'])),
 ('01.271.2839.9.061',
  'ENC.PREV.C/O REGIME GERAL DE PREVIDÊNCIA',
  810,
  868,
  (['3.1.90.13.00', '2.100.000,00'], ['3.1.90.92.00', '200,00'])),
 ('01.272.2839.9.001',
  'ENC.PREV. C/O REGIME PRÓPRIO DE PREVIDÊNCIA',
  928,
  989,
  (['3.1.91.13.00', '1.580.000,00'],
   ['3.1.91.92.00', '200,00'],
   ['3.3.90.08.00', '200,00'

In [20]:
count = 0
for orgao, unidade, despesas in unidades:
    o_obj, created = models.Orgao.objects.get_or_create(
        loa=loa, codigo=orgao[0], especificacao=orgao[1]
    )

    uo_obj, created = models.UnidadeOrcamentaria.objects.get_or_create(
        loa=loa, orgao=o_obj, codigo=unidade[0], especificacao=unidade[1]
    )
    
    for desp, espec, x1, x2, naturezas in despesas:
        desp_re = re.match(r'(?P<funcao>\d{2})\.(?P<subfuncao>\d{3})'
                           r'\.(?P<programa>\d{4})\.(?P<acao>\d\.\d{3})', desp)

        funcao_obj = models.Funcao.objects.filter(loa=loa, codigo=desp_re['funcao']).first()
        subfunc_obj = models.SubFuncao.objects.filter(loa=loa, codigo=desp_re['subfuncao']).first()

        programa_obj, created = models.Programa.objects.get_or_create(loa=loa, codigo=desp_re['programa'])
        acao_obj, created = models.Acao.objects.get_or_create(loa=loa, codigo=desp_re['acao'], especificacao=espec)
        
        for nat in naturezas:
            count += 1
            natureza, created = models.Natureza.objects.get_or_create(loa=loa, codigo=nat[0])
            
            desp_obj, created = models.Despesa.objects.get_or_create(
                loa = loa,
                orgao = o_obj,
                unidade = uo_obj,
                funcao = funcao_obj,
                subfuncao = subfunc_obj,
                programa = programa_obj,
                acao = acao_obj,
                natureza = natureza
            )
            
            desp_obj.valor_materia = Decimal(nat[1].replace('.', '').replace(',', '.'))
            
            try: 
                desp_obj.save()
            except Exception as e:
                print(e)
                break
        if not desp_obj.pk:
            break
print(count, models.Despesa.objects.filter(loa=loa).count())

1093 1093


In [21]:
a = """text_global = load_pages_with(page_mask='ADENDO III A PORTARIA SOF N.15 DE 20/06/78')
naturezas = []
for m in re.finditer(r'\n?(\d{1,2}\.\d{1,2}\.\d{2}\.\d{2}\.\d{2})\n([^0-9]*)?\n?', text_global):
    naturezas.append(m.groups())
naturezas = sorted(naturezas, key=lambda x: x[0])
for esp, cod in naturezas:
    nat, created = models.Natureza.objects.get_or_create(loa=loa, codigo=cod)
    nat.especificacao=esp
    nat.save()
    """

In [22]:
text_global = load_pages_with(page_mask='ADENDO III A PORTARIA SOF N.08 DE 04/02/85')
naturezas = []
for m in re.finditer(r'\n(\d{1,2}\.\d{1,2}\.\d{2}\.\d{2}\.\d{2})\n(.*)\n', text_global):
    naturezas.append(m.groups())
naturezas = sorted(naturezas, key=lambda x: x[0])

In [23]:
text_global[0:1]

'E'

In [24]:
for cod, esp in naturezas:
    nat, created = models.Natureza.objects.get_or_create(loa=loa, codigo=cod)
    nat.especificacao=esp
    nat.save()

In [25]:
models.Natureza.objects.filter(loa=loa, codigo__startswith='7.0.00').update(especificacao='')

1

In [26]:
with open(f'{loa.ano}_natureza_despesa.txt', 'r') as fnd:
    lines = fnd.read()
    lines = re.split(r'\t|\n', lines)
    naturezas = [(lines[i], lines[i+1]) for i in range(0, len(lines)-1, 2)] 
    for cod, esp in naturezas:
        nat, created = models.Natureza.objects.get_or_create(loa=loa, codigo=cod)
        if not nat.especificacao:
            nat.especificacao=esp
            nat.save()  

In [27]:
text_global = load_pages_with(page_mask='ADENDO V A PORTARIA SOF N.15 DE 20/06/78')

In [28]:
programas = set()
for m in re.finditer(r'\n(\d{4})\n(.+)\n', text_global):
    programas.add(m.groups())

In [29]:
programas

{('0139', 'PROCESSO LEGISLATIVO'),
 ('0239', 'APOIO AO PODER JUDICIÁRIO'),
 ('0439', 'PROGRAMA GESTÃO MODERNA E TRANSFORMADORA'),
 ('0639', 'DEFESA SOCIAL E SEGURANCA PUBLICA PRESENTE'),
 ('0839', 'JATAÍ INCLUSÃO SOCIAL A TODOS'),
 ('0939', 'GESTÃO DA SEGURIDADE SOCIAL DOS SERVIDORES'),
 ('1039', 'ASSISTÊNCIA A SAÚDE'),
 ('1239', 'EDUCAÇÃO EM FOCO'),
 ('1339', 'ARTE E CIDADANIA'),
 ('1539', 'AVANÇO NAS MELHORIAS DOS SERVIÇOS DE DESENVOLVIMENTO URBANO'),
 ('1639', 'FAMÍLIA HABITAÇÃO DIGNA'),
 ('1839', 'POLÍTICAS AMBIENTAIS E A MELHORIA NA QUALIDADE DE VIDA'),
 ('1939', 'PROGRAMA DE APOIO À CIÊNCIA, TECNOLOGIA E INOVAÇÃO'),
 ('2039', 'PROGRAMA DE INFRAESTRUTURA E APOIO TÉCNICO AO MORADOR E PRODUTOR'),
 ('2239', 'PROGRAMA DE DESENVOLVIMENTO ECONÔMICO DE JATAÍ'),
 ('2339', 'TURISMO EM AVANÇO'),
 ('2639', 'TRANSPORTES'),
 ('2739', 'VALORIZAÇÃO DO ESPORTE E LAZER'),
 ('2839', 'ENCARGOS ESPECIAIS'),
 ('7999', 'RESERVA TÉCNICA DO RPPS'),
 ('9999', 'RESERVA DE CONTIGENCIA')}

In [30]:
len(programas), models.Programa.objects.filter(loa=loa).values_list('codigo', flat=True).count()

(21, 21)

In [31]:
for cod, esp in list(programas):
    prog, created = models.Programa.objects.get_or_create(loa=loa, codigo=cod)
    prog.especificacao=esp
    prog.save()

In [32]:
models.Despesa.objects.order_by('-id').filter(loa=loa).values()[:2]

<QuerySet [{'id': 1093, 'loa_id': 2024, 'orgao_id': 15, 'unidade_id': 32, 'funcao_id': 17, 'subfuncao_id': 63, 'programa_id': 11, 'acao_id': 281, 'fonte_id': None, 'natureza_id': 1, 'valor_materia': Decimal('1350000.00'), 'valor_norma': Decimal('0.00')}, {'id': 1092, 'loa_id': 2024, 'orgao_id': 14, 'unidade_id': 31, 'funcao_id': 4, 'subfuncao_id': 8, 'programa_id': 3, 'acao_id': 280, 'fonte_id': None, 'natureza_id': 32, 'valor_materia': Decimal('900.00'), 'valor_norma': Decimal('0.00')}]>

In [33]:
#for desp in models.Despesa.objects.order_by('-id').filter(acao_id=1531):
#    print(desp.orgao.codigo, ' - ', desp.orgao.especificacao, f"'{desp.natureza.codigo}'", desp.valor_norma == desp.valor_materia)

In [34]:
#for desp in models.Despesa.objects.order_by('-id').exclude(valor_materia=F('valor_norma')):
#    print(desp, f"'{desp.natureza.codigo}'", desp.valor_norma, desp.valor_materia)

In [35]:
soma_total = models.Despesa.objects.filter(loa=loa).aggregate(vm=Sum('valor_materia'), vn=Sum('valor_norma'))

In [36]:
soma_total, soma_total['vm'] - soma_total['vn']

({'vm': Decimal('728401000.00'), 'vn': Decimal('0.00')},
 Decimal('728401000.00'))

In [37]:
for soma in models.Despesa.objects.filter(loa=loa).values(
    'orgao__codigo', 'unidade__codigo'
).order_by('orgao__codigo', 'unidade__codigo').annotate(vm=Sum('valor_materia'), vn=Sum('valor_norma')):
    #del soma['orgao__codigo']
    print(soma)

{'orgao__codigo': '01', 'unidade__codigo': '01', 'vm': Decimal('25850000.00'), 'vn': Decimal('0.00')}
{'orgao__codigo': '01', 'unidade__codigo': '02', 'vm': Decimal('150000.00'), 'vn': Decimal('0.00')}
{'orgao__codigo': '03', 'unidade__codigo': '02', 'vm': Decimal('16411357.29'), 'vn': Decimal('0.00')}
{'orgao__codigo': '03', 'unidade__codigo': '03', 'vm': Decimal('49690252.98'), 'vn': Decimal('0.00')}
{'orgao__codigo': '03', 'unidade__codigo': '06', 'vm': Decimal('29768611.33'), 'vn': Decimal('0.00')}
{'orgao__codigo': '03', 'unidade__codigo': '08', 'vm': Decimal('92471609.38'), 'vn': Decimal('0.00')}
{'orgao__codigo': '03', 'unidade__codigo': '10', 'vm': Decimal('6127425.98'), 'vn': Decimal('0.00')}
{'orgao__codigo': '03', 'unidade__codigo': '11', 'vm': Decimal('115030504.23'), 'vn': Decimal('0.00')}
{'orgao__codigo': '03', 'unidade__codigo': '15', 'vm': Decimal('10214887.40'), 'vn': Decimal('0.00')}
{'orgao__codigo': '03', 'unidade__codigo': '16', 'vm': Decimal('8689755.38'), 'vn': 

In [38]:
# mostra unidades que totais mudaram da matéria para a norma
sp = 0
sn = 0
for soma in models.Despesa.objects.filter(loa=loa).values(
    'orgao__codigo', 'unidade__codigo'
).order_by('orgao__codigo', 'unidade__codigo').annotate(vm=Sum('valor_materia'), vn=Sum('valor_norma')):
    if soma['vm'] - soma['vn']:
        del soma['orgao__codigo']
        s = soma['vm'] - soma['vn']
        
        if s > 0:
            sp += s
        else:
            sn += s
    
        print(soma,soma['vn'] - soma['vm'])
    #print(soma['unidade__codigo'], soma['vm'] - soma['vn'])
print(sp, sn) #11.035.815,25

{'unidade__codigo': '01', 'vm': Decimal('25850000.00'), 'vn': Decimal('0.00')} -25850000.00
{'unidade__codigo': '02', 'vm': Decimal('150000.00'), 'vn': Decimal('0.00')} -150000.00
{'unidade__codigo': '02', 'vm': Decimal('16411357.29'), 'vn': Decimal('0.00')} -16411357.29
{'unidade__codigo': '03', 'vm': Decimal('49690252.98'), 'vn': Decimal('0.00')} -49690252.98
{'unidade__codigo': '06', 'vm': Decimal('29768611.33'), 'vn': Decimal('0.00')} -29768611.33
{'unidade__codigo': '08', 'vm': Decimal('92471609.38'), 'vn': Decimal('0.00')} -92471609.38
{'unidade__codigo': '10', 'vm': Decimal('6127425.98'), 'vn': Decimal('0.00')} -6127425.98
{'unidade__codigo': '11', 'vm': Decimal('115030504.23'), 'vn': Decimal('0.00')} -115030504.23
{'unidade__codigo': '15', 'vm': Decimal('10214887.40'), 'vn': Decimal('0.00')} -10214887.40
{'unidade__codigo': '16', 'vm': Decimal('8689755.38'), 'vn': Decimal('0.00')} -8689755.38
{'unidade__codigo': '32', 'vm': Decimal('1333583.70'), 'vn': Decimal('0.00')} -1333583

In [39]:
for soma in models.Despesa.objects.filter(loa=loa).values(
    'orgao__codigo', 
).order_by('orgao__codigo', 'unidade__codigo', 'acao__codigo'
          ).annotate(vm=Sum('valor_materia'), vn=Sum('valor_norma')
                    ):
    if soma['vm'] == soma['vn']:
        continue
    del soma['orgao__codigo']
    #del soma['unidade__codigo']
    #rint(soma['acao__codigo'], soma['vm'] - soma['vn'])

In [40]:
#mostra despesas de uma unidade
for soma in models.Despesa.objects.filter(orgao__codigo='01').values(
    'acao__codigo', 'natureza__codigo', 'valor_materia', 'valor_norma'
):
    del soma['acao__codigo']
    #del soma['valor_materia']
    print(soma) #35 - 116.778,08  #36 - 202.657,51 

{'natureza__codigo': '4.4.90.51.00', 'valor_materia': Decimal('1487025.43'), 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '4.4.90.52.00', 'valor_materia': Decimal('1070000.00'), 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '3.1.90.11.00', 'valor_materia': Decimal('17500000.00'), 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '3.1.90.16.00', 'valor_materia': Decimal('4100.00'), 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '3.1.90.96.00', 'valor_materia': Decimal('70000.00'), 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '3.3.90.14.00', 'valor_materia': Decimal('75000.00'), 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '3.3.90.30.00', 'valor_materia': Decimal('660000.00'), 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '3.3.90.33.00', 'valor_materia': Decimal('1000.00'), 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '3.3.90.34.00', 'valor_materia': Decimal('2500.00'), 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '3.3.90.35.

In [41]:
for soma in models.Despesa.objects.filter(loa=loa, orgao__codigo='01').values(
    'subfuncao__codigo', 'subfuncao__especificacao'
).order_by('subfuncao__codigo').annotate(vm=Sum('valor_materia'), vn=Sum('valor_norma'), alt=Sum('registrocontabil_set__valor')):
    #del soma['subfuncao__codigo']
    print(soma)

{'subfuncao__codigo': '031', 'subfuncao__especificacao': 'AÇÃO LEGISLATIVA', 'vm': Decimal('22055325.43'), 'vn': Decimal('0.00'), 'alt': None}
{'subfuncao__codigo': '271', 'subfuncao__especificacao': 'PREVIDÊNCIA BÁSICA', 'vm': Decimal('2100200.00'), 'vn': Decimal('0.00'), 'alt': None}
{'subfuncao__codigo': '272', 'subfuncao__especificacao': 'PREVIDÊNCIA DO REGIME ESTATUTÁRIO', 'vm': Decimal('1580600.00'), 'vn': Decimal('0.00'), 'alt': None}
{'subfuncao__codigo': '846', 'subfuncao__especificacao': 'OUTROS ENCARGOS ESPECIAIS', 'vm': Decimal('263874.57'), 'vn': Decimal('0.00'), 'alt': None}


In [42]:
for soma in models.Despesa.objects.filter(loa=loa, orgao__codigo='01', unidade__codigo='02'):
    print(soma)

01.031.0139.1.149:4.4.90.52.00 - OBRAS E EQUIPAMENTOS DA ESCOLA DO LEGISLATIVO
01.031.0139.2.101:3.3.90.14.00 - MANUTENÇÃO DE ATIVIDADES DA ESCOLA DO LEGISLATIVO
01.031.0139.2.101:3.3.90.30.00 - MANUTENÇÃO DE ATIVIDADES DA ESCOLA DO LEGISLATIVO
01.031.0139.2.101:3.3.90.36.00 - MANUTENÇÃO DE ATIVIDADES DA ESCOLA DO LEGISLATIVO
01.031.0139.2.101:3.3.90.39.00 - MANUTENÇÃO DE ATIVIDADES DA ESCOLA DO LEGISLATIVO


In [43]:
Acao.objects.filter(loa__ano=2024,).count()

281

In [44]:
result = run_sql('''SELECT
  SUBSTRING(n.codigo, 1, 1) AS nat,
  
  sum(d.valor_materia)
                    from loa_despesa d
                        inner join loa_loa        l on (d.loa_id = l.id)
                        inner join loa_natureza   n on (d.natureza_id = n.id)
                    group by nat
                    order by nat

;''')
result

[('3', Decimal('490145667.67')),
 ('4', Decimal('217346646.43')),
 ('7', Decimal('14142322.39')),
 ('9', Decimal('6766363.51'))]

In [45]:
for r in result[:10]:
    print(r)

('3', Decimal('490145667.67'))
('4', Decimal('217346646.43'))
('7', Decimal('14142322.39'))
('9', Decimal('6766363.51'))


In [46]:
for u in models.UnidadeOrcamentaria.objects.filter(codigo='02'):
    #p.save()
    print(u, u.orgao)

02 - ESCOLA DO LEGISLATIVO 01 - CAMARA MUNICIPAL DE JATAI
02 - GABINETE DO PREFEITO 03 - PREFEITURA MUNICIPAL DE JATAI
